In [1]:
%matplotlib inline
%load_ext line_profiler

from tiptop.tiptop import *

Cupy import successfull. Installed version is: 8.5.0
Cupy import successfull. Installed version is: 8.5.0


In [2]:
fullInputPath = "perfTest"
parametersFile = "HarmoniLTAO_1"
#parametersFileGPU = 'params1GPU'
#parametersFileCPU = 'params1CPU'
baseOutpuPath = 'perfTest'
outputFile = "test"

In [4]:
# %lprun -f MavisLO.computeTotalResidualMatrix 
overallSimulation(fullInputPath, parametersFile, baseOutpuPath, outputFile, doPlot=True, doConvolve=True)

SyntaxError: invalid syntax (<string>, line 1)

In [ ]:
%%time
overallSimulation(fullInputPath, parametersFileGPU, baseOutpuPath, outputFile, doConvolve=True, pitchScaling = 0.9)

In [ ]:
%%time
overallSimulation(fullInputPath, parametersFileCPU, baseOutpuPath, outputFile, doConvolve=True, pitchScaling = 0.9)

In [ ]:
%lprun -f MavisLO.computeTotalResidualMatrix overallSimulation(fullInputPath, parametersFileCPU, baseOutpuPath, outputFile, doConvolve=True, pitchScaling = 0.9)

In [ ]:
# MAVIS SIMULATIONS
baseInputPath = '/data7/guido/results/MAVIS/res4psf/20201116_111006.0_oaPSFs_w'
baseOutpuPath = '/data4/frossi/'

wvlPaths = ['450','550','640','890']
k=-1
for wvlPath in wvlPaths:
    k+=1
    fullInputPath = baseInputPath + wvlPath + 'nm/'
    parametersFile = 'params'
    outputFile = 'simulation' + wvlPath + 'nm'
    overallSimulation(fullInputPath, parametersFile, baseOutpuPath, outputFile,doConvolve=True,pitchScaling = 0.9)
print('Done')